In [1]:
import torch

from layers.graph_transformer_edge_layer import MultiHeadAttentionLayer
print(torch.__version__)
print(torch.version.cuda)
import dgl



Using backend: pytorch


1.6.0
10.2


In [54]:
# Step 2: Create a sample graph
src_nodes = torch.tensor([0, 1, 2])
dst_nodes = torch.tensor([1, 2, 3])
g = dgl.graph((src_nodes, dst_nodes))

# Step 3: Define input node and edge features
num_nodes = g.num_nodes()
num_edges = g.num_edges()
in_dim = 4
out_dim = 1
num_heads = 2
use_bias = True

node_features = torch.rand((num_nodes, in_dim))  # Random node features
edge_features = torch.rand((num_edges, in_dim))  # Random edge features

g.ndata['Q'] = torch.tensor([[1],[2],[3],[4]])
g.ndata['K'] = torch.tensor([[100],[200],[300],[400]])


torch.Size([4, 1])

In [63]:
def src_dot_dst(src_field, dst_field, out_field):
    def func(edges):
        print((edges.src[src_field] * edges.dst[dst_field]).sum(-1))
        return {out_field: (edges.src[src_field] * edges.dst[dst_field]).sum(-1, keepdim=True)}
    return func
g.apply_edges(src_dot_dst('Q', 'K', 'Out'))

tensor([ 200,  600, 1200])


In [63]:
multihead_layer = MultiHeadAttentionLayer(in_dim=in_dim, out_dim=out_dim, num_heads=num_heads, use_bias=use_bias)
h_out, e_out = multihead_layer(g, node_features, edge_features)


print("Output Node Features (h_out):")
print(h_out)
print("\nOutput Edge Features (e_out):")
print(e_out)

Output Node Features (h_out):
tensor([[[ 0.0000],
         [ 0.0000]],

        [[ 0.3369],
         [-0.4612]],

        [[ 0.2504],
         [-0.3839]],

        [[ 0.3835],
         [ 0.0638]]], grad_fn=<DivBackward0>)

Output Edge Features (e_out):
tensor([[[0.3323],
         [0.0359]],

        [[0.1370],
         [0.0285]],

        [[0.2942],
         [0.0137]]], grad_fn=<MulBackward0>)
